In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets
import torchvision.transforms as transforms

In [2]:
class NN(nn.Module):
    #here, num_classes = number of labels
    def __init__(self,input_size,num_classes):
        super(NN,self).__init__()
        self.fc1 = nn.Linear(input_size,50)
        self.fc2 = nn.Linear(50,num_classes)
    
    def forward(self,x):
        #here, x is my input,it is passed as parameter to the nn.Linear
        #nn.Linear expects the input so that it can apply forward pass on it.
        #note that self.fc1 is variable referencing to a nn.Linear layer,so anything passed to it 
        #is reflected to the nn.Linear
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [3]:
#more explanation of the above thing
def sayHello(text):
    return text.upper()
print(sayHello('Hello'))
#I can also do
hello = sayHello
print(hello('Hello'))

HELLO
HELLO


In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:
#Hyperparameters
input_size=784
num_classes=10
learning_rate=0.001
batch_size=64
num_epochs = 1

In [6]:
# loading Data
#H x W x C <- C x H x W
train_dataset = datasets.MNIST(root='dataset/',train=True,transform=transforms.ToTensor(),download=True)
train_loader = DataLoader(dataset=train_dataset,batch_size=batch_size,shuffle=True)
test_dataset = datasets.MNIST(root='dataset/',train=False,transform=transforms.ToTensor(),download=True)
test_loader = DataLoader(dataset=test_dataset,batch_size=batch_size,shuffle=True)

Extracting dataset/MNIST/raw/train-images-idx3-ubyte.gz to dataset/MNIST/raw


Extracting dataset/MNIST/raw/train-labels-idx1-ubyte.gz to dataset/MNIST/raw


Extracting dataset/MNIST/raw/t10k-images-idx3-ubyte.gz to dataset/MNIST/raw


Extracting dataset/MNIST/raw/t10k-labels-idx1-ubyte.gz to dataset/MNIST/raw
Processing...
Done!


/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/torchvision/datasets/mnist.py:469: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ../torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [7]:
#initialization of the network
model = NN(input_size=input_size,num_classes=num_classes).to(device)

In [11]:
#loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr=learning_rate)

In [24]:
for batch,(data,targets) in enumerate(train_loader):
    data = data.to(device=device)
    print(data.shape[0])
    data = data.reshape(data.shape[0],-1)
    print(data.shape[0])
    
    data = data.reshape(-1,data.shape[0])
    print(data.shape[0])
    
    data = data.view(data.shape[0],-1)
    print(data.shape[0])
    break

64
64
784
784


In [35]:
#Train Network
for epoch in range(num_epochs):
    for batch_idx,(data,targets) in enumerate(train_loader):
        data = data.to(device=device)
        targets = targets.to(device=device)
        
        #get the correct shape
        data = data.view(data.shape[0],-1)
        
        #forward
#         scores = model(data)
        scores = model.forward(data)
        #training loss
        loss = criterion(scores,targets)
        
        #backward
        optimizer.zero_grad()
        loss.backward()
        
        #optimizing the parameters
        optimizer.step()
        
    
#Checking the accuracy of the model on training and test set 
def check_accuracy(loader,model):
    if loader.dataset.train:
        print("Checking accuracy on training data")
    else:
        print("Checking accuracy on test data")
    
    num_correct = 0
    num_samples = 0
    model.eval()
    
    with torch.no_grad():
        for x, y in loader:
#             x = x.to(device=device)
#             y = y.to(devivce=device)
            x = x.view(x.shape[0],-1)
            
            scores = model(x)
            # here, max(1) looks for max value on each row, from top to bottom
            #eg, [0,0,0,0,0,5,0,0,0,0] returns 5
            _,predictions = scores.max(1)
            print(predictions)
            print(predictions.size())
            num_correct +=(predictions == y).sum()
            num_samples += predictions.size(0)
            break
        print(f'Got {num_correct} / {num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}')
    model.train()

check_accuracy(train_loader,model)
check_accuracy(test_loader,model)

Checking accuracy on training data
tensor([6, 3, 2, 5, 2, 9, 9, 2, 7, 4, 8, 9, 0, 3, 8, 0, 5, 1, 5, 6, 8, 2, 3, 7,
        0, 2, 0, 0, 3, 3, 1, 2, 4, 1, 8, 5, 2, 7, 5, 5, 0, 3, 3, 7, 5, 1, 5, 6,
        5, 8, 7, 7, 1, 6, 5, 0, 8, 2, 9, 2, 8, 6, 9, 0])
torch.Size([64])
Got 64 / 64 with accuracy 100.00
Checking accuracy on test data
tensor([4, 9, 3, 3, 6, 2, 0, 5, 7, 1, 2, 1, 4, 0, 4, 3, 7, 4, 9, 9, 2, 9, 8, 2,
        3, 7, 7, 7, 3, 5, 9, 3, 9, 1, 5, 0, 8, 0, 2, 5, 1, 6, 0, 1, 8, 3, 4, 0,
        9, 2, 0, 4, 3, 0, 9, 3, 3, 3, 4, 9, 5, 8, 4, 0])
torch.Size([64])
Got 61 / 64 with accuracy 95.31


In [33]:
a = torch.tensor([[1,2,3,4],[1,9,3,7]])
a.max(1)

torch.return_types.max(
values=tensor([4, 9]),
indices=tensor([3, 1]))